# Imports 

In [10]:
import pandas as pd
import pickle
import time

datapath = '../Data/'

startTime = time.time()

# Get Data

In [11]:
filename = 'processed.pickle'
infile = open(datapath+filename,'rb')
df = pickle.load(infile)
infile.close()
df.head()

,SPA_ACCT_ID,SPA_PREM_ID,MONTH,RES_EL_CUR120_DAYS,RES_EL_CUR22_DAYS,RES_EL_CUR30_DAYS,RES_EL_CUR60_DAYS,RES_EL_CUR90_DAYS,RES_EL_CUR_BAL_AMT,RES_EL_OVER_120_DAYS,...,TOTAL_30_DAYS_AMT,TOTAL_60_DAYS_AMT,TOTAL_90_DAYS_AMT,TOTAL_CUR_BALANCE,LAST_MO_W_DATA,WITHIN_6_MO_PRIOR_LAST_DATA,MO_AWAY,NUM_PREM_FOR_PER,NUM_PER_FOR_PREM,MULTI_DWELL_SIZE
0,6,1557,0,0.0,66.03,0.0,0.0,0.0,66.03,0.0,...,0.00,0.0,0.0,331.78,False,False,NaN,1.0,1.0,52212
1,6,1557,1,0.0,61.26,0.0,0.0,0.0,61.26,0.0,...,139.70,0.0,0.0,442.93,False,False,NaN,1.0,1.0,52212
2,6,1557,2,0.0,59.74,0.0,0.0,0.0,59.74,0.0,...,0.00,0.0,0.0,273.26,False,False,NaN,1.0,1.0,52212
3,6,1557,3,0.0,59.74,0.0,0.0,0.0,59.74,0.0,...,0.00,0.0,0.0,380.90,False,False,NaN,1.0,1.0,52212
4,6,1557,4,0.0,52.47,0.0,0.0,0.0,52.47,0.0,...,118.82,0.0,0.0,356.45,False,False,NaN,1.0,1.0,52212


In [12]:
df.columns.to_list()

['SPA_ACCT_ID',
 'SPA_PREM_ID',
 'MONTH',
 'RES_EL_CUR120_DAYS',
 'RES_EL_CUR22_DAYS',
 'RES_EL_CUR30_DAYS',
 'RES_EL_CUR60_DAYS',
 'RES_EL_CUR90_DAYS',
 'RES_EL_CUR_BAL_AMT',
 'RES_EL_OVER_120_DAYS',
 'RES_GAS_CUR120_DAYS',
 'RES_GAS_CUR22_DAYS',
 'RES_GAS_CUR30_DAYS',
 'RES_GAS_CUR60_DAYS',
 'RES_GAS_CUR90_DAYS',
 'RES_GAS_CUR_BAL_AMT',
 'RES_GAS_OVER_120_DAYS',
 'BREAK_ARRANGEMENT',
 'BREAK_PAY_PLAN',
 'CALL_OUT',
 'CALL_OUT_MANUAL',
 'DUE_DATE',
 'FINAL_NOTICE',
 'PAST_DUE',
 'SEVERANCE_ELECTRIC',
 'SEVERANCE_GAS',
 'CITY_TOT_DUE',
 'CITY_30_DAYS_PAST_DUE_AMT',
 'CITY_60_DAYS_PAST_DUE_AMT',
 'CITY_90_DAYS_PAST_DUE_AMT',
 'SPA_PER_ID',
 'CMIS_MATCH',
 'APARTMENT',
 'ENROLL_DATE',
 'HAS_COTENANT',
 'BLOCKGROUP_GEOID',
 'POSTAL',
 'MULTI_DWELL_ID',
 'AGG_INCOME_GEO',
 'NO_EARNINGS_GEO',
 'BELOW_POVERTY_LVL_GEO',
 'SNAP_GEO',
 'ABOVE_GRD7_GEO',
 'PUBLIC_ASSIST_GEO',
 'AVISTA_CUR120_DAYS',
 'AVISTA_OVER_120_DAYS',
 'AVISTA_CUR22_DAYS',
 'AVISTA_CUR30_DAYS',
 'AVISTA_CUR60_DAYS',
 'AVIST

# Select Subset of Attributes

In [13]:
keep = [
    'MONTH',
    #
    #'RES_EL_CUR30_DAYS',
    #'RES_EL_CUR60_DAYS',
    #'RES_EL_CUR90_DAYS',
    #'RES_EL_CUR_BAL_AMT',
    #'RES_GAS_CUR30_DAYS',
    #'RES_GAS_CUR60_DAYS',
    #'RES_GAS_CUR90_DAYS',
    #'RES_GAS_CUR_BAL_AMT',
    #
    'BREAK_ARRANGEMENT',
    'BREAK_PAY_PLAN',
    'CALL_OUT',
    'CALL_OUT_MANUAL',
    'DUE_DATE',
    'FINAL_NOTICE',
    'PAST_DUE',
    'SEVERANCE_ELECTRIC',
    'SEVERANCE_GAS',
    #
    #'CITY_TOT_DUE',
    #'CITY_30_DAYS_PAST_DUE_AMT',
    #'CITY_60_DAYS_PAST_DUE_AMT',
    #'CITY_90_DAYS_PAST_DUE_AMT',
    #
    'TOTAL_30_DAYS_AMT',
    'TOTAL_60_DAYS_AMT',
    'TOTAL_90_DAYS_AMT',
    'TOTAL_CUR_BALANCE',
    #
    'SPA_PER_ID',
    'CMIS_MATCH',
    #
    'HAS_COTENANT',
    'NUM_PREM_FOR_PER',
    'NUM_PER_FOR_PREM',
    'MULTI_DWELL_SIZE',
    #
    #'AGG_INCOME_GEO',
    #'NO_EARNINGS_GEO',
    #'BELOW_POVERTY_LVL_GEO',
    #'SNAP_GEO',
    #'ABOVE_GRD7_GEO',
    #'PUBLIC_ASSIST_GEO',
]

df = df[keep]
df.columns.to_list()

['MONTH',
 'BREAK_ARRANGEMENT',
 'BREAK_PAY_PLAN',
 'CALL_OUT',
 'CALL_OUT_MANUAL',
 'DUE_DATE',
 'FINAL_NOTICE',
 'PAST_DUE',
 'SEVERANCE_ELECTRIC',
 'SEVERANCE_GAS',
 'TOTAL_30_DAYS_AMT',
 'TOTAL_60_DAYS_AMT',
 'TOTAL_90_DAYS_AMT',
 'TOTAL_CUR_BALANCE',
 'SPA_PER_ID',
 'CMIS_MATCH',
 'HAS_COTENANT',
 'NUM_PREM_FOR_PER',
 'NUM_PER_FOR_PREM',
 'MULTI_DWELL_SIZE']

# Check

In [14]:
df.isnull().sum()

MONTH                 0
BREAK_ARRANGEMENT     0
BREAK_PAY_PLAN        0
CALL_OUT              0
CALL_OUT_MANUAL       0
DUE_DATE              0
FINAL_NOTICE          0
PAST_DUE              0
SEVERANCE_ELECTRIC    0
SEVERANCE_GAS         0
TOTAL_30_DAYS_AMT     0
TOTAL_60_DAYS_AMT     0
TOTAL_90_DAYS_AMT     0
TOTAL_CUR_BALANCE     0
SPA_PER_ID            0
CMIS_MATCH            0
HAS_COTENANT          0
NUM_PREM_FOR_PER      0
NUM_PER_FOR_PREM      0
MULTI_DWELL_SIZE      0
dtype: int64

In [15]:
df.dtypes

MONTH                   int64
BREAK_ARRANGEMENT       int32
BREAK_PAY_PLAN          int32
CALL_OUT                int32
CALL_OUT_MANUAL         int32
DUE_DATE                int32
FINAL_NOTICE            int32
PAST_DUE                int32
SEVERANCE_ELECTRIC      int32
SEVERANCE_GAS           int32
TOTAL_30_DAYS_AMT     float64
TOTAL_60_DAYS_AMT     float64
TOTAL_90_DAYS_AMT     float64
TOTAL_CUR_BALANCE     float64
SPA_PER_ID              int32
CMIS_MATCH               bool
HAS_COTENANT             bool
NUM_PREM_FOR_PER      float64
NUM_PER_FOR_PREM      float64
MULTI_DWELL_SIZE        int64
dtype: object

## Normalize Data

In [16]:
#

# Time and Save

In [17]:
filename = 'ann_ready.pickle'
outfile = open(datapath+filename, 'wb')
pickle.dump(df, outfile)
outfile.close()

In [18]:
from custom_methods.calc_time import calc_time_from_sec

calc_time_from_sec(time.time()-startTime)

hours:minutes:seconds = 0:0:5.7041308879852295
